# Titanic - Machine Learning from Disaster

## Imports

In [2]:
import opendatasets as od

In [3]:
import pandas as pd

In [4]:
import numpy as np

## 1. Getting the Data

In [5]:
!pip install opendatasets

In [6]:
od.download("https://www.kaggle.com/competitions/titanic/data")


Skipping, found downloaded files in ".\titanic" (use force=True to force download)


In [7]:
train_set = pd.read_csv('titanic/train.csv')

In [8]:
test_set = pd.read_csv('titanic/test.csv')

In [9]:
survived = pd.read_csv('titanic/gender_submission.csv')

In [10]:
train_set.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 2. Studying the Data

In [26]:
print('Empty rows:')

for category in train_set:
    print(category +  ": " + str(len(np.where(pd.isnull(train_set[category]))[0])))

Empty rows:
PassengerId: 0
Survived: 0
Pclass: 0
Name: 0
Sex: 0
Age: 177
SibSp: 0
Parch: 0
Ticket: 0
Fare: 0
Cabin: 687
Embarked: 2
